In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn import svm
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [ ]:
dataset = pd.read_csv('../input/factors-affecting-campus-placement/Placement_Data_Full_Class.csv')

In [ ]:
dataset.head()

In [ ]:
# as salary and sl_no columns are not required for placement status prediction so we drop it
dataset.drop(['salary','sl_no'], axis=1, inplace=True)

In [ ]:

# missing values checking
dataset.isnull().sum()

In [ ]:
# checking column values data type
dataset.info()

Label Encoding Data

In [ ]:
# label encoding needs to be done to ensure all values in the dataset is numeric
# hsc_s, degree_t columns needs to be splitted into columns (get_dummies needs to be applied)
features_to_split = ['hsc_s','degree_t']
for feature in features_to_split:
    dummy = pd.get_dummies(dataset[feature])
    dataset = pd.concat([dataset, dummy], axis=1)
    dataset.drop(feature, axis=1, inplace=True)

In [ ]:
dataset

In [ ]:
dataset.rename(columns={"Others": "Other_Degree"},inplace=True)

In [ ]:
dataset

In [ ]:
encoder = LabelEncoder() # to encode string to the values like 0,1,2 etc.

In [ ]:

columns_to_encode = ['gender','ssc_b', 'hsc_b','workex','specialisation','status']
for column in columns_to_encode:
    dataset[column] = encoder.fit_transform(dataset[column])

In [ ]:

dataset

In [ ]:
dataset.describe()

Checking for Outliers

In [ ]:
fig, axs = plt.subplots(ncols=6,nrows=3,figsize=(20,10))
index = 0
axs = axs.flatten()
for k,v in dataset.items():
    sns.boxplot(y=v, ax=axs[index])
    index+=1

fig.delaxes(axs[index])
plt.tight_layout(pad=0.3, w_pad=0.5,h_pad = 4.5) # for styling by giving padding


In [ ]:

# deleting some outliers in 2 columns degree_p and hsc_p
dataset = dataset[~(dataset['degree_p']>=90)]
dataset = dataset[~(dataset['hsc_p']>=95)]

Checking for Correlation

In [ ]:
dataset.corr()

In [ ]:
# heatmap for checking correlation or linearity

plt.figure(figsize=(20,10))
sns.heatmap(dataset.corr().abs(), annot=True)

Correlation between the features are atmost 0.9 so they are not multi-correlated

In [ ]:

dataset.shape

In [ ]:
# checking distributions of all features
fig, axs = plt.subplots(ncols=6,nrows=3,figsize=(20,10))
index = 0
axs = axs.flatten()
for k,v in dataset.items():
    sns.distplot(v, ax=axs[index])
    index+=1

fig.delaxes(axs[index]) # deleting the 18th figure
plt.tight_layout(pad=0.3, w_pad=0.2,h_pad = 4.5)

In [ ]:

x = dataset.loc[:,dataset.columns!='status'] # all features are used
y = dataset.loc[:, 'status'] # label is status of placement

In [ ]:
x

In [ ]:
y

In [ ]:
sc= StandardScaler()
x_scaled = sc.fit_transform(x) # for standardising the features
x_scaled = pd.DataFrame(x_scaled)

In [ ]:
x_train,x_test, y_train, y_test = train_test_split(x_scaled,y,test_size=0.18, random_state=0)


Using Logistic Regression

In [ ]:
lr = LogisticRegression()

In [ ]:
lr.fit(x_train, y_train)

In [ ]:
y_pred = lr.predict(x_test)

In [ ]:
y_pred

In [ ]:
y_test

In [ ]:

accuracy_score(y_test, y_pred)

In [ ]:
lr.score(x_train,y_train)

In [ ]:
confusion_matrix(y_test, y_pred)

In [ ]:
print(classification_report(y_test,y_pred))